In [0]:
from fastai.vision import * 
###################################################Do this VVVVVVVVVVVVVVVVVVVVVV
path = Path('<path to the dataset>') # specify the path to your dataset

df = pd.read_csv(path/'train_v2.csv') ################################rename to correct csv
print(df.head())

In [0]:
tfms = get_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.05, max_warp=0.)

In [0]:
np.random.seed(42) # set random seed so we always get the same validation set
src = (ImageItemList.from_csv(path, 'train_v2.csv', folder='train-jpg', suffix='.jpg') ####################change names
       # Load data from csv
       .random_split_by_pct(0.2)
       # split data into training and validation set (20% validation)
       .label_from_df(label_delim=' ') #######################################################make sure this is correct
       # label data using the tags column (second column is default)
)

In [0]:
# for testing only
data.show_batch(rows=3, figsize=(12, 9))

In [0]:
# create metrics
acc_02 = partial(accuracy_thresh, thresh=0.2)
f_score = partial(fbeta, thresh=0.2)
# create cnn with the resnet50 architecture
learn = create_cnn(data, models.resnet50, metrics=[acc_02, f_score])

In [0]:
learn.lr_find() # find learning rate
learn.recorder.plot() # plot learning rate

The next few code blocks trains the thing in a few different ways.

In [0]:
lr = 0.01 ################## chosen learning rate based on code block above
learn.fit_one_cycle(4, lr) # train model for 4 epochs 

learn.save('multisave1') ###################maybe think of better name

In [0]:
learn.unfreeze() # unfreeze all layers

learn.lr_find() # find learning rate
learn.recorder.plot() # plot learning rate

In [0]:
#################################### Possibly change learning rate VVVVVVVVVVVVVVVVVVVVV
learn.fit_one_cycle(5, slice(1e-5, lr/5)) # fit model with differential learning rates

learn.save('multisave2') # save model

In [0]:
# switch resolution
data = (src.transform(tfms, size=256)
        .databunch(bs=16).normalize(imagenet_stats))
learn.data = data

In [0]:
learn.freeze() # freeze bottom layers again

learn.lr_find()
learn.recorder.plot()

In [0]:
lr=1e-2/2 #################################### change based on plot
learn.fit_one_cycle(5, lr)

learn.save('multisave3')

In [0]:
learn.unfreeze()

learn.lr_find()
learn.recorder.plot()

In [0]:
learn.fit_one_cycle(5, slice(1e-5, lr/5)) ######################### change based on plot

learn.save('multisave4')

In [0]:
learn.export()